Import relevant libraries 

In [1]:
import pandas as pd

Read data from csv file

In [2]:
reviews = pd.read_csv('../data/kenya_hotel_reviews.csv')

reviews.head()

,id,name,hotel,review,date,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15
0,2,APPRECIATION,Golden Tulip Westlands Nairobi,Am very greatfull and thankfull to Golden Tuli...,August 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3,Best time at tulip fitness centre,Golden Tulip Westlands Nairobi,"The gym is well equiped, clean and I love the ...",August 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,4,"Golden Tulip, Westlands",Golden Tulip Westlands Nairobi,My stay at Golden Tulip was wonderful. The ser...,August 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5,Appriciation,Golden Tulip Westlands Nairobi,"This comes as the best of all times ,being my ...",July 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6,A must visit,Golden Tulip Westlands Nairobi,As part of the most reliable and convenient pl...,July 2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
reviews = reviews[['name', 'hotel', 'review', 'date']]
reviews.head()

,name,hotel,review,date
0,APPRECIATION,Golden Tulip Westlands Nairobi,Am very greatfull and thankfull to Golden Tuli...,August 2021
1,Best time at tulip fitness centre,Golden Tulip Westlands Nairobi,"The gym is well equiped, clean and I love the ...",August 2021
2,"Golden Tulip, Westlands",Golden Tulip Westlands Nairobi,My stay at Golden Tulip was wonderful. The ser...,August 2021
3,Appriciation,Golden Tulip Westlands Nairobi,"This comes as the best of all times ,being my ...",July 2021
4,A must visit,Golden Tulip Westlands Nairobi,As part of the most reliable and convenient pl...,July 2021


In [4]:
import re

# remove punctuation
# reviews['review_processed'] = reviews['review'].map(lambda x: re.sub('[,\.!?]', '', x))

# convert all text to lower case
# reviews['review_processed'] = reviews['review'].map(lambda x: x.lower())

reviews['review_processed'] = reviews['review']
reviews['review_processed'].head()

0    Am very greatfull and thankfull to Golden Tuli...
1    The gym is well equiped, clean and I love the ...
2    My stay at Golden Tulip was wonderful. The ser...
3    This comes as the best of all times ,being my ...
4    As part of the most reliable and convenient pl...
Name: review_processed, dtype: object

In [5]:
import gensim
from gensim.utils import simple_preprocess
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords


[nltk_data] Downloading package stopwords to
[nltk_data]     /home/shadowfox/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
        
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) 
             if word not in stop_words] for doc in texts]


data = reviews.review_processed.values.tolist()
data_words = list(sent_to_words(data))

data_words = remove_stopwords(data_words)
# print(data_words[:1][0][:30])
print(data_words)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [8]:
import gensim.corpora as corpora
# Create Dictionary
id2word = corpora.Dictionary(data_words)
# Create Corpus
texts = data_words
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
# View
print(corpus[:1][0][:30])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 3), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 3), (16, 1), (17, 3), (18, 1), (19, 1), (20, 3)]


In [9]:
from pprint import pprint
# number of topics
num_topics = 5
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=num_topics)
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.024*"hotel" + 0.014*"staff" + 0.012*"food" + 0.011*"place" + '
  '0.011*"service" + 0.010*"room" + 0.010*"nice" + 0.010*"great" + '
  '0.009*"good" + 0.009*"stay"'),
 (1,
  '0.023*"hotel" + 0.017*"good" + 0.016*"great" + 0.015*"staff" + '
  '0.014*"service" + 0.011*"food" + 0.009*"rooms" + 0.008*"friendly" + '
  '0.008*"restaurant" + 0.008*"room"'),
 (2,
  '0.020*"staff" + 0.013*"food" + 0.013*"good" + 0.013*"service" + '
  '0.012*"hotel" + 0.012*"rooms" + 0.011*"great" + 0.010*"room" + 0.010*"stay" '
  '+ 0.009*"friendly"'),
 (3,
  '0.035*"hotel" + 0.019*"good" + 0.018*"staff" + 0.014*"rooms" + 0.013*"stay" '
  '+ 0.012*"room" + 0.012*"food" + 0.010*"great" + 0.009*"service" + '
  '0.007*"well"'),
 (4,
  '0.019*"hotel" + 0.016*"staff" + 0.015*"good" + 0.014*"rooms" + 0.012*"room" '
  '+ 0.009*"excellent" + 0.009*"great" + 0.009*"friendly" + 0.008*"food" + '
  '0.008*"stayed"')]


In [10]:
import os
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_data_filepath = os.path.join('../reports/ldavis_prepared_'+str(num_topics))
# # this is a bit time consuming - make the if statement True
# # if you want to execute visualization prep yourself
if 1 == 1:
    LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
    with open(LDAvis_data_filepath, 'wb') as f:
        pickle.dump(LDAvis_prepared, f)
# load the pre-prepared pyLDAvis data from disk
with open(LDAvis_data_filepath, 'rb') as f:
    LDAvis_prepared = pickle.load(f)
pyLDAvis.save_html(LDAvis_prepared, '../reports/ldavis_prepared_'+ str(num_topics) +'.html')
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.005533 -0.010868       1        1  36.837324
1     -0.004750 -0.023353       2        1  18.039181
0     -0.006304  0.003417       3        1  17.823672
4      0.030773  0.013362       4        1  16.169454
2     -0.025252  0.017442       5        1  11.130369, topic_info=          Term         Freq         Total Category  logprob  loglift
168      staff  8225.000000   8225.000000  Default  30.0000  30.0000
15     service  5057.000000   5057.000000  Default  29.0000  29.0000
401  excellent  3038.000000   3038.000000  Default  28.0000  28.0000
86       place  3666.000000   3666.000000  Default  27.0000  27.0000
154      great  5498.000000   5498.000000  Default  26.0000  26.0000
..         ...          ...           ...      ...      ...      ...
81       hotel   691.072497  12857.231301   Topic5  -4.3848  -0.7279
126         us   256.968549   2003.652989   Topic5  -5.3740   0.1417
502       nice   331.261217   3467.579540   Topic5  -5.1201  -0.1528
10     nairobi   268.948719   3412.801982   Topic5  -5.3285  -0.3453
91        time   235.488652   2680.766034   Topic5  -5.4613  -0.2367

[409 rows x 6 columns], token_table=       Topic      Freq      Term
term                            
17754      1  0.163580      abdi
17754      2  0.817902      abdi
17523      1  0.884771  acoustic
17523      4  0.068059  acoustic
17781      3  0.884414     adada
...      ...       ...       ...
174        4  0.144977     would
174        5  0.119954     would
8038       2  0.072040        yo
8038       3  0.864480        yo
8038       4  0.072040        yo

[899 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 1, 5, 3])